In [1]:
import pandas as pd
import string
import re
pd.get_option('max_columns')
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
data = pd.read_csv('Suicide_Detection.csv')
data2 = data.iloc[:1000,]
data2.shape

(1000, 3)

In [2]:
def transform_text(text):
    lem = WordNetLemmatizer()
    text = text.lower()
    text = nltk.word_tokenize(text)

    a = []
    for i in text:
        if i.isalnum():
            a.append(i)

    text = a[:]
    a.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            a.append(i)

    text = a[:]
    a.clear()

    for i in text:
        a.append(lem.lemmatize(i))

    return " ".join(a)





In [4]:
transform_text('hello my name is sushie')

'hello name sushie'

In [6]:
data2['text_transformed']=data2['text'].apply(transform_text)

C:\Users\dell\AppData\Local\Temp/ipykernel_8320/1531507128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['text_transformed']=data2['text'].apply(transform_text)


In [7]:
data2.head()

,Unnamed: 0,text,class,text_transformed
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide,ex wife threatening suiciderecently left wife ...
1,3,Am I weird I don't get affected by compliments...,non-suicide,weird get affected compliment coming someone k...
2,4,Finally 2020 is almost over... So I can never ...,non-suicide,finally 2020 almost never hear 2020 bad year e...
3,8,i need helpjust help me im crying so hard,suicide,need helpjust help im cry hard
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,losthello name adam 16 struggling year afraid ...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x=cv.fit_transform(data2['text_transformed']).toarray()

In [9]:
x.shape


(1000, 7610)

In [23]:
y = pd.get_dummies(data2['class'])
y = y.iloc[:,:1]
y


,non-suicide
0,0
1,1
2,1
3,0
4,0
...,...
995,0
996,1
997,1
998,1


In [21]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state=0)

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(x_train, y_train)


c:\users\dell\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
prediction = model.predict(x_test)

In [27]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(prediction, y_test)

acc

0.83

In [29]:
import pickle as pl
pl.dump(cv, open('vectorixer.pkl', 'wb'))
pl.dump(model, open('pan.pkl', 'wb'))